In this notebook we demostrate how to use BioBERT , which is BERT pre trained on a huge corpus of medical data. We will demonstrate text classification. Tasks such as NER and slot filling can be easily performed by replacing the pre-trained model with the pre-trainied biobert.
<br><br>
Now to make use of the pre-trained bioBert model with the hugging face transformers library we need the model's weights to be in a form which pyTorch understands. The original model was trained using tensorflow so we need to convert the weights into pyTorch weights. We can then import and use it.


First we will download the model from github repo of [BioBert](https://github.com/dmis-lab/biobert).<br> Huge shoutout to this [article](https://medium.com/@acpanjan/download-google-drive-files-using-wget-3c2c025a8b99) which helped in using wget to download the pre-trained model.

Download the mtsamples.csv dataset which can be found [here](https://www.kaggle.com/tboyle10/medicaltranscriptions)

In [ ]:
## To install only the requirements of this notebook, uncomment the lines below and run this cell
!pip install requests==2.23.0
!pip install pytorch-transformers==1.2.0
!pip install transformers==4.7.0
!pip install pandas==1.1.5
!pip install pytorch-pretrained-bert==0.6.2
!pip install pytorch-nlp==0.5.0
!pip install tensorflow #==1.14.0
!pip install torch #==1.10.0
!pip install scikit-learn==0.21.3
!pip install tqdm #==4.41.1
!pip install matplotlib==3.2.2
## To install the requirements for the entire chapter, uncomment the lines below and run this cell
# try:
#   import google.colab
#   !curl  https://raw.githubusercontent.com/practical-nlp/practical-nlp/master/Ch10/ch10-requirements.txt | xargs -n 1 -L 1 pip install
# except ModuleNotFoundError:
#   !pip install -r "ch10-requirements.txt"

     |████████████████████████████████| 176 kB 13.8 MB/s 
     |████████████████████████████████| 895 kB 49.8 MB/s 
     |████████████████████████████████| 1.2 MB 37.0 MB/s 
     |████████████████████████████████| 131 kB 49.8 MB/s 
     |████████████████████████████████| 8.4 MB 34.0 MB/s 
     |████████████████████████████████| 79 kB 7.8 MB/s 
     |████████████████████████████████| 138 kB 53.7 MB/s 
     |████████████████████████████████| 127 kB 50.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 2.5 MB 11.1 MB/s 
     |████████████████████████████████| 3.3 MB 46.9 MB

In [ ]:
#import packages
import requests
import tarfile
import os
import shutil
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) #ignore warnings

#define functions to download bioBert from google drive if wget doesn't work
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

#download bioBert
try:
  import google.colab
  !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD" -O biobert_weights && rm -rf /tmp/cookies.txt
except ModuleNotFoundError:
  print("Using Google Drive\n")
  file_id = '1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD'
  destination = 'Data/biobert_weights'
  download_file_from_google_drive(file_id, destination)

--2021-12-10 02:19:22--  https://docs.google.com/uc?export=download&confirm=YW_E&id=1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD
Resolving docs.google.com (docs.google.com)... 142.251.5.102, 142.251.5.113, 142.251.5.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.5.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-7c-docs.googleusercontent.com/docs/securesc/t2v6fb7dj329qsp6ovuuld9mtc3dmfnr/i8pi775fdn2reo4ib04ute62el0gekb3/1639102725000/13799006341648886493/14421948226659402872Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download [following]
--2021-12-10 02:19:22--  https://doc-0o-7c-docs.googleusercontent.com/docs/securesc/t2v6fb7dj329qsp6ovuuld9mtc3dmfnr/i8pi775fdn2reo4ib04ute62el0gekb3/1639102725000/13799006341648886493/14421948226659402872Z/1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD?e=download
Resolving doc-0o-7c-docs.googleusercontent.com (doc-0o-7c-docs.googleusercontent.com)... 142.251.5.132, 2a00:1450:400c:c1b::84
Connecting to d

Install the required libraries

Unzip the file and convert the weights to a pytorch readable form. Stefan-it's solution to the issue [here](https://github.com/huggingface/transformers/issues/457) was extremely helpful.

In [ ]:
try:
  print("part 1")
  import google.colab
  !tar -xzf biobert_weights
  !ls biobert_v1.1_pubmed/
except ModuleNotFoundError:
  tar = tarfile.open('Data/biobert_weights','r')
  tar.extractall('Data/')
  tar.close()
  print(os.listdir('Data/biobert_v1.1_pubmed'))

PATH = "."
try:
  import google.colab
  PATH = './'
except ModuleNotFoundError:
  PATH = 'Data/'

try:
  print("part 2")
  import google.colab
  !transformers-cli convert --model_type bert --tf_checkpoint biobert_v1.1_pubmed/model.ckpt-1000000 --config biobert_v1.1_pubmed/bert_config.json --pytorch_dump_output biobert_v1.1_pubmed/pytorch_model.bin
except ModuleNotFoundError:
  !transformers-cli convert --model_type bert --tf_checkpoint Data/biobert_v1.1_pubmed/model.ckpt-1000000 --config Data/biobert_v1.1_pubmed/bert_config.json --pytorch_dump_output Data/biobert_v1.1_pubmed/pytorch_model.bin

# try:
#   print("part 3")
#   import google.colab 
#   !ls biobert_v1.1_pubmed/
#   !mv biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/config.json
#   !ls biobert_v1.1_pubmed/
# except ModuleNotFoundError:
#   print(os.listdir(PATH+'biobert_v1.1_pubmed'))
#   shutil.copy(PATH+'biobert_v1.1_pubmed/bert_config.json',PATH+'biobert_v1.1_pubmed/config.json')
#   print(os.listdir(PATH+'biobert_v1.1_pubmed'))

In [ ]:
# #importing the dataset
try:
  print('Using Wget')
  !wget https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch10/Data/mtsamples.csv -O mtsamples.csv 
except ModuleNotFoundError:
  from google.colab import files
  uploaded = files.upload()

import pandas as pd
df = pd.read_csv(PATH+'mtsamples.csv')
df.head()

Using Wget
--2021-12-10 02:19:50--  https://raw.githubusercontent.com/practical-nlp/practical-nlp-code/master/Ch10/Data/mtsamples.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17007236 (16M) [text/plain]
Saving to: ‘mtsamples.csv’

mtsamples.csv       100%[===================>]  16.22M  --.-KB/s    in 0.1s    

2021-12-10 02:19:51 (153 MB/s) - ‘mtsamples.csv’ saved [17007236/17007236]



,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
print(df.shape)
df['medical_specialty'].value_counts()

(4999, 6)


 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Dentistry                          27
 Cosmetic / Plastic Surge

lets try predicting the medical_speciality from the description
dataset which is highly imbalanced. Could remove ones less which are less than 5%. But as it is a demonstration let us just proceed with how the dataset it and look at the results

From here it will be the same as the IMDB_sentiment_classification (ch4) notebook so we will not re-explain every step.

In [ ]:
#importing a few necessary packages and setting the DATA directory
DATA_DIR="."
import os
import numpy as np
import pickle
import tensorflow as tf
 

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    n_gpu = torch.cuda.device_count()
    torch.cuda.get_device_name(0)

from sklearn.preprocessing import LabelEncoder

#Encoding medical_specialty
le = LabelEncoder()
df["medical_specialty"] = le.fit_transform(df["medical_specialty"])
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,0,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,2,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,2,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,3,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,3,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [ ]:
description = list(df['description'])
# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(PATH+'biobert_v1.1_pubmed', do_lower_case=True)

# Restricting the max size of Tokens to 512(BERT doest accept any more than this)
tokenized_texts = list(map(lambda t: ['[CLS]']+tokenizer.tokenize(t)+['[SEP]'] , description))
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

classes = list(df['medical_specialty'])

Tokenize the first sentence:
['[CLS]', 'a', '23', '-', 'year', '-', 'old', 'white', 'female', 'presents', 'with', 'complaint', 'of', 'all', '##er', '##gies', '.', '[SEP]']


In [ ]:
# Set the maximum sequence length. 
MAX_LEN = 128

# Pad our input tokens so that everything has a uniform length
input_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, tokenized_texts)),
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
batch_size = 16

# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, classes, 
                                                            random_state=2020, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2020, test_size=0.1)
                                             
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

#Loading pre trained BERT
#import model
from pytorch_transformers import BertModel
# model = BertModel.from_pretrained(PATH+'biobert_v1.1_pubmed')
model = BertForSequenceClassification.from_pretrained(PATH+'biobert_v1.1_pubmed', num_labels=40)#binary classification

if torch.cuda.is_available():
    print(model.cuda())
else:
    print(model)

In [1]:
# BERT fine-tuning parameters

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

torch.cuda.empty_cache() 
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 2#4

# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
    # Set our model to training mode
    model.train()  
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  ## VALIDATION

    # Put model in evaluation mode
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

# plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

NameError: ignored

The model performs very bad in terms of accuracy. Pre-processing the data and hyper parameter tuning will help us to achieve better results.